In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from skimage.filters import threshold_mean
import sys

sys.path.append('../')
from src.io import read_parameters

In [6]:
parameter_file = "../conf/local/parameters.yml"

parameters = read_parameters(parameter_file)

base_folder = parameters["base_folder"]
output_folder = parameters["output_folder"]
key_file_path = parameters["key_file_path"]

plot_width = 7
plot_height = 7
channel_artery = 1

key_file = pd.read_csv(key_file_path)
print(key_file.columns)
display(key_file)

#filename_GFPxERG = base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Erg X GFP.tif"
#filename_ERG = base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Erg_threshold.ome.tif"
#filename_drawn =  base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Drawn.ome.tiff"

Index(['Unnamed: 0', 'filename', 'ExperimentID', 'VEGFR3CreERT2 mTmG',
       'VEGFR3CreERT2;mTmG;Cdc42Flox/Flox', 'VEGFR3CreERT2;mTmG;Rac1Flox/Flox',
       'Drawn', 'GFP X ERG', 'GFP', 'ERG', 'IB4', 'THO Injection Point',
       'Collection Point', 'None', 'Moderate', 'Strong', 'Large veins',
       'Thin arteries', 'Small radial expansion', 'Hypersprouting',
       'Large sprouting front', 'Deep layer sprouting failure',
       'Retina with problems', 'Pixel size in um', 'Analyzed by Silvanus',
       'Longest vessel', 'Average length', 'Heterozygous', 'Homozygous',
       'condition', 'time_interval', 'start_time', 'end_time', 'delta_time'],
      dtype='object')


,Unnamed: 0,filename,ExperimentID,VEGFR3CreERT2 mTmG,VEGFR3CreERT2;mTmG;Cdc42Flox/Flox,VEGFR3CreERT2;mTmG;Rac1Flox/Flox,Drawn,GFP X ERG,GFP,ERG,...,Analyzed by Silvanus,Longest vessel,Average length,Heterozygous,Homozygous,condition,time_interval,start_time,end_time,delta_time
0,0,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,1,0,0,0,...,1.0,1660.143,1185.8080,na,na,control,P5_P6,5.0,6.0,1.0
1,1,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,1,0,0,...,1.0,1660.143,1185.8080,na,na,control,P5_P6,5.0,6.0,1.0
2,2,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,0,1,0,...,1.0,1660.143,1185.8080,na,na,control,P5_P6,5.0,6.0,1.0
3,3,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,0,0,1,...,1.0,1660.143,1185.8080,na,na,control,P5_P6,5.0,6.0,1.0
4,4,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,2,1,0,0,1,0,0,0,...,1.0,1611.684,1239.1413,na,na,control,P5_P6,5.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,1016,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,237,0,1,0,1,0,0,0,...,0.0,NaN,NaN,NaN,NaN,Cdc42,P5_P6,5.0,6.0,1.0
946,1017,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,237,0,1,0,0,1,0,0,...,0.0,NaN,NaN,NaN,NaN,Cdc42,P5_P6,5.0,6.0,1.0
947,1018,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,237,0,1,0,0,0,1,0,...,0.0,NaN,NaN,NaN,NaN,Cdc42,P5_P6,5.0,6.0,1.0
948,1019,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,237,0,1,0,0,0,0,1,...,0.0,NaN,NaN,NaN,NaN,Cdc42,P5_P6,5.0,6.0,1.0


In [ ]:
summary_data = pd.DataFrame()
counter = 0

experiment_IDs = key_file["ExperimentID"].unique() #[70:]
#experiment_IDs = [10]

for experiment_ID in experiment_IDs:
    print(experiment_ID)
    if experiment_ID in experiment_IDs_exclude:
        print("Excluded experiment ID: %s" % experiment_ID)
        continue
    experiment_key = key_file[key_file["ExperimentID"] == experiment_ID]
    
    
    
    display(experiment_key)
    filename_ERG = base_folder
    filename_GFPxERG = base_folder
    filename_drawn = base_folder
    
    count_img = 0
    for index, row in experiment_key.iterrows():
        
        if row["ERG"] == 1:
            filename_ERG = base_folder + row["filename"]
            count_img += 1
        if row["GFP X ERG"] == 1:
            filename_GFPxERG = base_folder + row["filename"]
            count_img += 1
        if row["Drawn"] == 1:
            filename_drawn =  base_folder + row["filename"]
            count_img += 1
            
    if count_img != 3:
        print("Need exactly 3 images, but %s were given!" % count_img)
        continue
    
    print(filename_ERG)
    print(filename_GFPxERG)
    print(filename_drawn)
    
    img_GFPxERG = np.array(io.imread(filename_GFPxERG))
    #print(img_GFPxERG.shape)
    #print(threshold_mean(img_GFPxERG))
    thresh_img_GFPxERG = img_GFPxERG > threshold_mean(img_GFPxERG)
    unique_GFPxERG, counts_GFPxERG = np.unique(thresh_img_GFPxERG, return_counts = True)    
    if counts_GFPxERG[1] > counts_GFPxERG[0]:
        thresh_img_GFPxERG = img_GFPxERG < threshold_mean(img_GFPxERG)
        unique_GFPxERG, counts_GFPxERG = np.unique(thresh_img_GFPxERG, return_counts = True)
        #counts_GFPxERG = np.count_nonzero(thresh_img_GFPxERG)
    
    #print(np.unique(img_GFPxERG))
    #print(np.unique(thresh_img_GFPxERG))
    img_ERG = np.array(io.imread(filename_ERG))
    #print(img_ERG.shape)
    #print(threshold_mean(img_ERG))
    thresh_img_ERG = img_ERG > threshold_mean(img_ERG)
    unique_ERG, counts_ERG = np.unique(thresh_img_ERG, return_counts = True) 
    if counts_ERG[1] > counts_ERG[0]:
        thresh_img_ERG = img_ERG < threshold_mean(img_ERG)
        unique_ERG, counts_ERG = np.unique(thresh_img_ERG, return_counts = True)

    img_drawn = np.array(io.imread(filename_drawn))
    print("Shape of drawn image:")
    print(img_drawn.shape)
    img_artery = img_drawn[1,:,:]
    #print(threshold_mean(img_artery))
    thresh_img_artery = img_artery > threshold_mean(img_artery)
    GFP_artery = np.logical_and(thresh_img_GFPxERG,thresh_img_artery)
    
    img_vein = img_drawn[2,:,:]
    #print(threshold_mean(img_artery))
    thresh_img_vein = img_vein > threshold_mean(img_vein)
    GFP_vein = np.logical_and(thresh_img_GFPxERG,thresh_img_vein)
    
    #unique_ERG, counts_ERG = np.unique(thresh_img_ERG, return_counts = True)
    #unique_GFP_artery, counts_GFP_artery = np.unique(GFP_artery, return_counts = True)
    #unique_GFP_vein, counts_GFP_vein = np.unique(GFP_vein, return_counts = True)
    
    counts_GFP_artery = np.count_nonzero(GFP_artery)
    counts_GFP_vein = np.count_nonzero(GFP_vein)
    
    
    #print(counts_ERG)
    #print(counts_GFPxERG)
    #print(100.0*counts_GFPxERG[1]/counts_ERG[1])
    #print(100.0*counts_GFP_artery[1]/counts_ERG[1])
    #print(100.0*counts_GFP_artery[1]/counts_GFPxERG[1])
    
    
    summary_data.at[counter,"ExperimentID"] = experiment_ID
    summary_data.at[counter,"condition"] = row["condition"]
    summary_data.at[counter,"time_interval"] = row["time_interval"]
    summary_data.at[counter,"ERG_px"] = counts_ERG[1]
    summary_data.at[counter,"GFPxERG_px"] = counts_GFPxERG[1]
    summary_data.at[counter,"GFPxERG_artery_px"] = counts_GFP_artery
    summary_data.at[counter,"GFPxERG_vein_px"] = counts_GFP_vein
    
    summary_data.to_csv(out_folder + "summary_data.csv", index=False)
    
    fig, ax = plt.subplots(3,2, figsize=(2*plot_width, 3*plot_height))
    ax[0,0].imshow(thresh_img_ERG)
    ax[0,1].imshow(thresh_img_GFPxERG)
    ax[1,0].imshow(thresh_img_artery)
    ax[1,1].imshow(GFP_artery)
    ax[2,0].imshow(thresh_img_vein)
    ax[2,1].imshow(GFP_vein)
    plt.savefig(out_folder + "validation_images/masks_experiment_id_%s.png" % experiment_ID)
    plt.close()
    
    counter += 1    

1


,filename,ExperimentID,VEGFR3CreERT2 mTmG,VEGFR3CreERT2;mTmG;Cdc42Flox/Flox,VEGFR3CreERT2;mTmG;Rac1Flox/Flox,Drawn,GFP X ERG,GFP,ERG,IB4,...,Analyzed by Silvanus,Longest vessel,Average length,Heterozygous,Homozygous,condition,time_interval,start_time,end_time,delta_time
0,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,1,0,0,0,NaN,...,1.0,1660.143,1185.808,na,na,control,P5_P6,5.0,6.0,1.0
1,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,1,0,0,NaN,...,1.0,1660.143,1185.808,na,na,control,P5_P6,5.0,6.0,1.0
2,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,0,1,0,NaN,...,1.0,1660.143,1185.808,na,na,control,P5_P6,5.0,6.0,1.0
3,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,1,1,0,0,0,0,0,1,NaN,...,1.0,1660.143,1185.808,na,na,control,P5_P6,5.0,6.0,1.0


/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1/Erg_threshold.ome.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1/Erg X GFP.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1/Drawn.ome.tiff
Shape of drawn image:
(7, 6144, 6144)
2


,filename,ExperimentID,VEGFR3CreERT2 mTmG,VEGFR3CreERT2;mTmG;Cdc42Flox/Flox,VEGFR3CreERT2;mTmG;Rac1Flox/Flox,Drawn,GFP X ERG,GFP,ERG,IB4,...,Analyzed by Silvanus,Longest vessel,Average length,Heterozygous,Homozygous,condition,time_interval,start_time,end_time,delta_time
4,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,2,1,0,0,1,0,0,0,NaN,...,1.0,1611.684,1239.1413,na,na,control,P5_P6,5.0,6.0,1.0
5,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,2,1,0,0,0,1,0,0,NaN,...,1.0,1611.684,1239.1413,na,na,control,P5_P6,5.0,6.0,1.0
6,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,2,1,0,0,0,0,1,0,NaN,...,1.0,1611.684,1239.1413,na,na,control,P5_P6,5.0,6.0,1.0
7,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,2,1,0,0,0,0,0,1,NaN,...,1.0,1611.684,1239.1413,na,na,control,P5_P6,5.0,6.0,1.0


/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1a/Erg_threshold.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1a/Erg X GFP.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/retina1a/Drawn.ome.tiff
Shape of drawn image:
(7, 6144, 6144)
3


...

/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #4 Retina 1/ERG.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #4 Retina 1/GFP X ERG.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #4 Retina 1/Drawn.ome.tiff
Shape of drawn image:
(7, 9318, 9318)
116


,filename,ExperimentID,VEGFR3CreERT2 mTmG,VEGFR3CreERT2;mTmG;Cdc42Flox/Flox,VEGFR3CreERT2;mTmG;Rac1Flox/Flox,Drawn,GFP X ERG,GFP,ERG,IB4,...,Analyzed by Silvanus,Longest vessel,Average length,Heterozygous,Homozygous,condition,time_interval,start_time,end_time,delta_time
376,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,116,1,0,0,1,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,control,P8_P9,8.0,9.0,1.0
377,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,116,1,0,0,0,1,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,control,P8_P9,8.0,9.0,1.0
378,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,116,1,0,0,0,0,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,control,P8_P9,8.0,9.0,1.0
379,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,116,1,0,0,0,0,0,1,NaN,...,0.0,NaN,NaN,NaN,NaN,control,P8_P9,8.0,9.0,1.0


/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #6 Retina 1/ERG.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #6 Retina 1/GFP X ERG.tif
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P8_P9/Vegfr3iCre_mTmG_B780_p8-p9/Animal #6 Retina 1/Drawn.ome.tiff
Shape of drawn image:
(7, 9318, 9318)
117


,filename,ExperimentID,VEGFR3CreERT2 mTmG,VEGFR3CreERT2;mTmG;Cdc42Flox/Flox,VEGFR3CreERT2;mTmG;Rac1Flox/Flox,Drawn,GFP X ERG,GFP,ERG,IB4,...,Analyzed by Silvanus,Longest vessel,Average length,Heterozygous,Homozygous,condition,time_interval,start_time,end_time,delta_time
380,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,117,1,0,0,1,0,0,0,0.0,...,0.0,NaN,NaN,NaN,NaN,control,P5_P6,5.0,6.0,1.0
381,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,117,1,0,0,0,1,0,0,0.0,...,0.0,NaN,NaN,NaN,NaN,control,P5_P6,5.0,6.0,1.0
382,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,117,1,0,0,0,0,1,0,0.0,...,0.0,NaN,NaN,NaN,NaN,control,P5_P6,5.0,6.0,1.0
383,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,117,1,0,0,0,0,0,1,0.0,...,0.0,NaN,NaN,NaN,NaN,control,P5_P6,5.0,6.0,1.0
384,/mouse_retina/data VEGFR3CRe and Rac and Cdc42...,117,1,0,0,0,0,0,0,1.0,...,0.0,NaN,NaN,NaN,NaN,control,P5_P6,5.0,6.0,1.0


/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/Vegfr3iCre_mTmG_B1032_p5-p6/Vegfr3iCre_mTmG_B1032_p5-p6_Animal#1_Retina#1/ERG.ome.tiff
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/Vegfr3iCre_mTmG_B1032_p5-p6/Vegfr3iCre_mTmG_B1032_p5-p6_Animal#1_Retina#1/GFPxERG.ome.tiff
/media/fjodor/TOSHIBA//mouse_retina/data VEGFR3CRe and Rac and Cdc42/VEGFR3Cre mTmG/P5_P6/Vegfr3iCre_mTmG_B1032_p5-p6/Vegfr3iCre_mTmG_B1032_p5-p6_Animal#1_Retina#1/Drawn.ome.tiff
